In [47]:

import plotly 
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.io as pio
import os


# for ploting online plots, account info is needed
# plotly.tools.set_credentials_file(username='ariellilili', api_key='X8MtW4k2mpBL5tUUHZdi')

# create offline plots
plotly.offline.init_notebook_mode(connected=True)

track = pd.read_csv('track_camera.csv')
track.head()

,timestamp,cameraID,from,to,stay
0,2019-05-10 10:00:03,2,中区1,进门区,558
1,2019-05-10 10:00:55,2,里间,中区2,416
2,2019-05-10 10:05:19,2,进门区,中区2,105
3,2019-05-10 10:06:50,2,进门区,中区2,99
4,2019-05-10 10:07:17,2,进门区,中区2,60


In [56]:
# bar plot - stay in each zone
df= track['stay'].groupby(track['from']).mean().to_frame().sort_values(by=['stay'],ascending=False)
# seconds to minutes
df = df/60
df['stay'] = df['stay'].round(1)

data = [go.Bar(
            x=df.index,
            y=df['stay'],
            text=df['stay'],
            textposition = 'inside',
    marker=dict(color='#4ea1d3'),
    opacity=0.8
        )]
layout =  dict(
    title = "各区域停留时间(分钟)",
    yaxis=dict(
        showticklabels=False,
        showgrid=False
    ),
    font=dict(family='微软雅黑')
)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)
pio.write_image(fig, '各区停留时间.jpeg', scale=5, width=600, height = 400)

In [52]:
# Sankey diagram
df = track.groupby(['from', 'to'],as_index=False)['timestamp'].count()
df = df[:8]
df['source_index'] = [1,1,1,2,2,2,0,0]
df['target_index'] = [4,3,6,5,3,6,1,2]

data = dict(
    type='sankey',
    node = dict(
      pad = 15,
      thickness = 20,
      label = ['进门区','中区1','中区2','进门区','中区2','中区1','里间'],
      color = ['#454552'] * 3 + ['#e85a71'] + ['#454552'] * 3
    ),
    link = dict(
      source = df['source_index'],
      target = df['target_index'],
      value = df['timestamp'],
    color = ['#d8e9ef'] * 8
  ))

# Setting up the layout settings in the "layout" argument
layout =  dict(
    title = "人流路线",
    font=dict(family='微软雅黑')
)

fig = dict(data=[data], layout=layout)
plotly.offline.iplot(fig)
pio.write_image(fig, '人流路线.jpeg', scale=5, width=600, height = 400)

In [58]:
# Set the datetime column as the index
track['timestamp'] = pd.to_datetime(track['timestamp'])
track.index = track['timestamp']
# Group the data by month, and take the mean for each group (i.e. each month)
df = track['stay'].groupby(track['from']).resample('H').count()
df = df.reset_index()
df.rename(columns={'stay':'count'}, inplace=True)

trace1 = go.Scatter(
    x=df[df['from'] == '进门区']['timestamp'],
    y=df[df['from'] == '进门区']['count'],
    name='进门区',
    mode = 'lines',
    marker=dict(color='#e85a71')
)
trace2 = go.Scatter(
    x=df[df['from'] == '中区1']['timestamp'],
    y=df[df['from'] == '中区1']['count'],
    name='中区1',
    mode = 'lines',
    marker=dict(color='#4ea1d3')
)
trace3 = go.Scatter(
    x=df[df['from'] == '中区2']['timestamp'],
    y=df[df['from'] == '中区2']['count'],
    name='中区2',
    mode = 'lines',
    marker=dict(color='#ffc952')
)
trace4 = go.Scatter(
    x=df[df['from'] == '里间']['timestamp'],
    y=df[df['from'] == '里间']['count'],
    name='里间',
    mode = 'lines',
    marker=dict(color='#454552')
)

data = [trace1, trace2,trace3,trace4]
layout = go.Layout(
    title="每小时在各区域人数",
    font=dict(family='微软雅黑')
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)
pio.write_image(fig, '各区人数.jpeg', scale=5, width=600, height = 400)